In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# How to normalize spectrograms
> Scaling spectrograms for neural networks.  

- toc: true 
- badges: true
- comments: true
- categories: [normalizing_spectrums]
- image: images/violin_spec.png

# Introduction  

Spectrograms are often treated as images in deep learning to leverage the many great techniques from image classification. A spectrogram, however, is fundamentally different from natural images in several ways as we will see below. That raises the central question of this post: what is the proper way to normalize spectrograms when training neural networks?

# Turning audio into an image

It is hard to overstate the success of deep neural networks for classifying images. This challenging task was previously dominated by expert handcrafted features. Now, the features are automatically learned from labeled data instead. The performance of these learned features completely shifted the Computer Vision paradigm. We would ideally like to follow these same, proven approaches in audio tasks.

However, audio is usually treated as a one dimensional signal in Machine Learning applications. Even stereo recordings with more than one channel are first mixed into mono (single channel) before processing. That means raw audio is unusable with 2D CNNs which are the bread and butter of modern image recognition. If we could represent audio in two dimensions, like an image, it opens up a host of deep learning image classification techniques.

Thankfully there are many ways to transform audio into two dimensions. The most common one is the short-time Fourier Transform [(STFT)](https://www.dsprelated.com/freebooks/sasp/Short_Time_Fourier_Transform.html). The STFT turns audio into a spectrogram: a 2-D signal representation in time and frequency. Since a spectrogram is two dimensional we can treat it like an image!  

Before plugging spectrograms into a neural network we need a data pipeline. Data normalization is one of the first steps in any good data pipeline. This is because learning is difficult with unnormalized inputs and convergence could take a very long time. For natural images, normalization involves: 
- Centering the image values by subtracting a single, scalar mean. 
- Dividing the images by a scalar standard deviation to give them a variance of 1.  

# Sample audio dataset


To keep things practical, we apply these normalization techniques to a [sound classification challenge](https://github.com/fastaudio/Audio-Competition) hosted by `fastaudio`. [`fastaudio`](https://github.com/fastaudio/fastaudio) is a community extension of the [`fastai`](https://github.com/fastai/fastai) library to make neural network audio tasks more accessible.  
The challenge here is to classify sounds in the [ESC-50 dataset](https://github.com/karolpiczak/ESC-50).
ESC-50 stands for "Environment Sound Classification with 50 classes". This set has a diverse set of sounds which gives a feel for how different audio spectrograms can be. Every file is five seconds long which makes batch processing easier since the samples are of the same size.

Many lines below are based on the `fastaudio` [baseline results notebook](https://github.com/fastaudio/Audio-Competition/blob/master/ESC-50-baseline-1Fold.ipynb).

## Importing `fastai` and `fastaudio` modules

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

## Downloading the ESC-50 dataset

The first step is downloading the ESC-50 dataset. It is included in `fastaudio` so we grab it with fastai's `untar_data`.

In [ ]:
# already in fastaudio, can download with fastai's `untar_data`
path = untar_data(URLs.ESC50)

Looking inside of the downloaded `audio` folder reveals many `.wav` files.  
Below we view the files with the `ls` method, a handy fastai addition to python's standard `pathlib.Path` class.

In [ ]:
wavs = (path/"audio").ls()
wavs

The output of `ls` shows there are 2,000 audio files. But the filenames are not too descriptive, so how can we know what is actually in each one?  
As with many datasets, the download includes a table with more information about the data (aka metadata).

In [ ]:
# read the audio metadata and show its first few rows
df = pd.read_csv(path/"meta"/"esc50.csv")
df.head()

The key information from the table is in the `filename` and `category` columns. The `filename` gives the name of the file inside of the `audio` folder. The `category` tells us which class it belongs to.

We pick the last file in the data directory as our working example. The file's `name` can index into the metadata table above to display its specific information.

In [ ]:
# pick the row where "filename" matches the waveform's "name".
df.loc[df.filename == wavs[-1].name]

This is a recording of crickets! We can load this audio file using the `create` function of the `AudioTensor` class in `fastaudio`. `AudioTensor` wraps a `torch.Tensor` with some extra syntactic sugar. 

In [ ]:
# create an AudioTensor from a file path
sample = AudioTensor.create(wavs[-1])

Thanks to `AudioTensor` we can directly plot and even listen to our sample. Each "burst" in the file is a cricket chirp. There are three full chirps and the early starts of a fourth chirp.

In [ ]:
sample.show()

# Normalizing waveforms

The first step is normalizing the audio waveform itself. We give it a mean of zero and unit variance in the typical way: 

$$\text{norm_audio} = \frac{\text{audio} - mean(\text{audio})}{std(\text{audio})} $$

In [ ]:
# normalize the waveform
norm_sample = (sample - sample.mean()) / sample.std()

Let's check if the mean is roughly 0 and the variance is roughly one:

In [ ]:
# checking if normalization worked
norm_sample.mean(),norm_sample.var()

Success! The waveform is normalized.

# Extracting spectrograms from audio

We are now ready to extract a spectrogram from the normalized audio. The `fastaudio` library wraps parts of [`torchaudio`](https://pytorch.org/audio/) to convert an `AudioTensor` into a spectrogram.

In [ ]:
# create a fastaudio Transform that converts audio into spectrograms
cfg = AudioConfig.BasicSpectrogram()
audio2spec = AudioToSpec.from_cfg(cfg)

The spectrogram computation details are not important here. But a quick look at the [spectrogram source code](https://pytorch.org/audio/_modules/torchaudio/functional.html#spectrogram) shows that it boils down to some pre and post processing around a [torch.stft](https://pytorch.org/docs/stable/generated/torch.stft.html) call. We can now transform our audio into a spectrogram.

In [ ]:
# extract the spectrogram
spec = audio2spec(norm_sample)

Just like with the waveform, `fastaudio` can directly plot spectrograms. The colorbar on the right is especially helpful here, since `matplotlib` always normalizes the values in a plot to a certain color range. Withtout the colorbar, the fixed color range makes it impossible to know or even guess the exact values in a spectrogram plot.

In [ ]:
spec.show();

This is a good time to compare the shapes of the audio vs. the spectrogram to see the added dimension that makes the spectrogram an "image".

In [ ]:
print(f'Audio shape [channels, samples]: {norm_sample.shape}')
print(f'Spectrum shape [channels, bins, time_steps]: {spec.shape}')

# How do we normalize spectrograms?

As stated in the introduction, a spectrogram is fundamentally different from an image.

Both dimensions in an image are in the spatial domain and have the same units.
Images are stored as integers in the range of `[0, 255]`.  
To normalize first divide all pixels by 255, the max possible value, to map them into `[0, 1]`.  
Then, find the statistics that approximately center the data and give it unit variance.  
For a color image we have three channels (RGB) and normalize each one.
If the image is grayscale then we normalize its single channel instead.
Given the layout of natural images, and the fact that both dimensions are in the same domain, it makes sense to normalize each channel with single, global values.

Spectrograms are different.
In a spectrogram, one dimension represents time and the other represents frequency.
Different quantities, scales, and sizes.
Frequency dimension given by choice of FFT size. Sets our spectral resolution.
Time dimension given by length of our signal, FFT size, and window overlap. Sets our temporal resolution.
In fact what we call the spectrogram is actually the log of the power spectrum.
Below we give a quick recap of how the spectrogram is computed to show how it differs from images.

If $\text{x}$ is the input audio then the STFT returns the spectrum:
$$\text{spectrum} = \text{STFT(x)}$$
We are more interested in the energy or power of the signal, so we take the absolute value of the STFT and square it:  
$$\text{power_spectrum} = |\text{STFT(x)}|^2$$ 
While we could use the power spectrum as our input "image", it is a bit problematic. The power spectrum often has a few, strong peaks and many small values. This means the values are [heavy-tailed](https://danielsdiscoveries.wordpress.com/2017/09/29/spectrogram-input-normalisation-for-neural-networks/) and make for poor network inputs.  
To deal with this we take the log of the power spectrum which spreads out the values. This becomes the spectrogram:
$$\text{spectrogram} = log(|\text{STFT(x)}|^2)$$
The range of the log function is from $-\infty$ to $+\infty$ which is very different than the integers from 0 to 255.


However, the spectrogram also introduces the notion of a different type of channel.
This makes "channel" an overloaded term for our purposes but it is still a crucial piece of the puzzle. 
The spectrogram transform can be interpreted as a "channelizer".
That is a fancy way to say that it takes the continuous frequency spectrum of our signal and chops it up into discrete bins, or channels. For example, consider a signal sampled at 16 kHz (typical for audio) where we take an STFT of size 512. Our spectrogram will have 512 channels where each one has a "bandwidth" of $$16 \ \text{kHz} \ \ / \ \ 512 \ \text{bins} = 31.25 \ \text{Hz per bin}$$

Even though these spectrum channels are different from the channels in an image, it raises the question: should we (or can we?) normalize an entire spectrum "image" with a single, global value? Or do we need to normalize each channel, as is done with images?  

There is no clear answer here, and your approach will likely depend both on the specifics of your problem and where your system will be deployed.
For example, if your are building a system that will be deployed in a similar environment as the training one, then it might make more sense to normalize by channels.
Your channel-based normalization statistics will follow the average noise floor and activity of the training data.
This motivation hold if you expect roughly the same patterns and distributions of activity once the system is deployed.
However, it will be critical to monitor the deployed environment and update the statistics as needed, else you slowly shift out of domain.

If your system will instead be used in a completely different environment, of which you have no knowledge, then the global statistics could be a better fit. While not as technically sound, your model won't be as surprised by radically new activity across different channels. 

Lastly, we also have issue of Transfer Learning. In Transfer Learning it is best-practice to normalize the new dataset with the statistics from the old dataset. In most cases that means normalizing with ImageNet statistics.
So if you are doing transfer learning, the easiest approach will be to use original stats. 
If your dataset is large enough that you are training from scratch, then the above applies.

# Spectrogram Normalization

Normalization statistics always come from the training set (this is a crucial place to avoid data leakage from the validation set). That means stepping through the training set once and getting a mean and standard deviation from each mini-batch. Then, we average the statistics from each mini-batch to get a pair of "global" statistics.  
One small detail: if your training dataset is large enough, you often do not need to iterate through the entire set. It is enough to sample 10% to 20% of the dataset for accurate statistics. However, since ESC-50 is small enough we get statistics from the whole set. 

First we to accumulate these statistics over mini-batches. We can borrow and slightly refactor a class from the incredibly helpful guide [here](http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.html).  
The `StatsRecorder` class below tracks the mean and standard deviation across mini-batches.

In [ ]:
class StatsRecorder:
    def __init__(self, red_dims=(0,2,3)):
        """Accumulates normalization statistics across mini-batches.
        ref: http://notmatthancock.github.io/2017/03/23/simple-batch-stat-updates.html
        """
        self.red_dims = red_dims # which mini-batch dimensions to average over
        self.nobservations = 0   # running number of observations

    def update(self, data):
        """
        data: ndarray, shape (nobservations, ndimensions)
        """
        # initialize stats and dimensions on first batch
        if self.nobservations == 0:
            self.mean = data.mean(dim=self.red_dims, keepdim=True)
            self.std  = data.std (dim=self.red_dims,keepdim=True)
            self.nobservations = data.shape[0]
            self.ndimensions   = data.shape[1]
        else:
            if data.shape[1] != self.ndimensions:
                raise ValueError('Data dims do not match previous observations.')
            
            # find mean of new mini batch
            newmean = data.mean(dim=self.red_dims, keepdim=True)
            newstd  = data.std(dim=self.red_dims, keepdim=True)
            
            # update number of observations
            m = self.nobservations * 1.0
            n = data.shape[0]

            # update running statistics
            tmp = self.mean
            self.mean = m/(m+n)*tmp + n/(m+n)*newmean
            self.std  = m/(m+n)*self.std**2 + n/(m+n)*newstd**2 +\
                        m*n/(m+n)**2 * (tmp - newmean)**2
            self.std  = torch.sqrt(self.std)
                                 
            # update total number of seen samples
            self.nobservations += n

By default `StatsRecorder` averages over the image channel dimensions (grayscale or RGB). The `red_dims` argument might look familiar from normalization code in other Computer Vision tasks (even the `Normalize` in `fastai`).  
Averaging instead over spectrogram channels is as easy as passing a different `red_dims`. 

## Building the dataset loader

Next we need to step through the training dataset. The setup belows follows the `fastaudio` ESC-50 baseline. One thing to mention is that by default, `fastaudio` resamples audio to 16 kHz. While much of the audio in the wild is sampled at 16 kHz, ESC-50 is actually sampled at a higher 44.1 kHz rate. Downsampling risks throwing away some information. But, keeping the higher sampling rate almost triples the "width" of the spectrogram. This much larger image potentially limits our batch size and architecture choices. For now we stick with downsampling to 16 kHz since it yields a very reasonable spectrogram shape of [201, 401].

We also need a `Transform` that normalizes individual audio waveforms.

In [ ]:
class AudioNormalize(Transform):
    "Normalizes a single `AudioTensor`."
    def encodes(self, x:AudioTensor): return (x-x.mean()) / x.std()

In [ ]:
def CrossValidationSplitter(col='fold', fold=1):
    "Split `items` (supposed to be a dataframe) by fold in `col`"
    def _inner(o):
        assert isinstance(o, pd.DataFrame), "ColSplitter only works when your items are a pandas DataFrame"
        col_values = o.iloc[:,col] if isinstance(col, int) else o[col]
        valid_idx = (col_values == fold).values.astype('bool')
        return IndexSplitter(mask2idxs(valid_idx))(o)
    return _inner

# do not resample audio
auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_x=ColReader("filename", pref=path/"audio"), 
                 splitter=CrossValidationSplitter(fold=1),
                 item_tfms = [AudioNormalize],
                 batch_tfms = [audio2spec],
                 get_y=ColReader("category"))
dbunch = auds.dataloaders(df, bs=64)
dbunch.show_batch(figsize=(7,7))

## Finding spectrogram normalization statistics

Below we create two statistic recorders: one for global statistics and one for channel-based statistics. Then we step through the entire dataset and find the normalization stats.

In [ ]:
# create recorders
global_stats  = StatsRecorder()
channel_stats = StatsRecorder(red_dims=(0,1,3))

# step through the dataset
with torch.no_grad():
    for idx,(x,y) in enumerate(iter(dbunch.train)):
        # update normalization statistics
        global_stats.update(x)
        channel_stats.update(x)
    
# parse out the stats
global_mean,global_std = global_stats.mean,global_stats.std
channel_mean,channel_std = channel_stats.mean,channel_stats.std

We can check the shape of both statistics to make sure they are as expected. For the global "grayscale" statistics, we expect a shape of: `[1,1,1,1]`. With spectrogram channel normalizations, we expect a shape of `[1,1,201,1]` with one normalization stat for each spectrogram bin.

In [ ]:
print('Shapes of global mean/std:')
global_mean.shape,global_std.shape 

In [ ]:
print('Shapes of channel mean/std:')
channel_mean.shape,channel_mean.shape 

# Making Normalization `Transforms`

We need to extend the `Normalize` in `fastai` to normalize the spectrogram mini-batches. The reason is type dispatch. `fastai` normalization uses ImageNet statistics due to the focus on transfer learning with color images. But this ImageNet normalization is only defined for the `TensorImage` class, while `AudioSpectrogram` subclasses the different `TensorImageBase`. The solution is to define `encodes` and `decodes` for `TensorImageBase` instead.

In [ ]:
class SpecNormalize(Normalize):
    "Normalize/denorm batch of `TensorImage`"
    def encodes(self, x:TensorImageBase): return (x-self.mean) / self.std
    def decodes(self, x:TensorImageBase):
        f = to_cpu if x.device.type=='cpu' else noop
        return (x*f(self.std) + f(self.mean))

In [ ]:
# make global and channel normalizers
GlobalSpecNorm  = SpecNormalize(global_mean,  global_std,  axes=(0,2,3))
ChannelSpecNorm = SpecNormalize(channel_mean, channel_std, axes=(0,1,3))

# Training with different statistics

Now for the moment of truth. We train with the two different spectrogram normalizations and measure their impact. For this we again follow the `fastaudio` baseline and train each type of normalization for 20 epochs. The final score is the averaged accuracy of five runs.

In [ ]:
epochs = 20
num_runs = 5

## Finding a good learning rate.

The learning rate is arguably the most critical neural network hyperparameter. The `lr_find` function in `fastai` is a great empirical way to set a good learning rate. The default learning rate in `cnn_learner` (0.001) is a good starting point. But since our task is so different from natural images it is worth re-evaluating this assumption.

In [ ]:
# auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
#                  get_x=ColReader("filename", pref=path/"audio"), 
#                  splitter=CrossValidationSplitter(fold=1), 
#                  item_tfms = [AudioNormalize], 
#                  batch_tfms = [audio2spec, GlobalSpecNorm],
#                  get_y=ColReader("category"))
# dbunch = auds.dataloaders(df, bs=64)

# learn = cnn_learner(dbunch, 
#                     xresnet18, 
#                     pretrained=False,
#                     config=cnn_config(n_in=1),
#                     loss_fn=CrossEntropyLossFlat,
#                     metrics=[accuracy],)
# learn.lr_find()

It seems we can make the learning rate higher than the default. 2e-3 looks like a good point in the curve, and we could potentially go even higher.

In [ ]:
good_lr = 2e-3 # from find_lr

## Training helpers  

To avoid repeating ourselves, the helper functions below will build dataloaders and run the training loops.  
The `get_dls` function makes it clear which normalization is being used. The `train_loops` function repeats training runs a given number of times.

In [ ]:
def get_dls(bs=64, item_tfms=[], batch_tfms=[]):
    "Get dataloaders with given `bs` and batch/item tfms."
    auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                     get_x=ColReader("filename", pref=path/"audio"), 
                     splitter=CrossValidationSplitter(fold=1),
                     item_tfms=item_tfms,   # for waveform normalization
                     batch_tfms=batch_tfms, # for spectrogram normalization
                     get_y=ColReader("category"))
    dls = auds.dataloaders(df, bs=bs)
    return dls

def train_loops(dls, name, num_runs=num_runs, epochs=epochs):
    "Runs `num_runs` training loops with `dls` for given `epochs`."
    accuracies = []
    for i in range(num_runs):
        learn = cnn_learner(dls, 
                            xresnet18, 
                            pretrained=False,
                            config=cnn_config(n_in=1),
                            loss_fn=CrossEntropyLossFlat,
                            metrics=[accuracy],)
        learn.fit_one_cycle(epochs, good_lr)
        accuracies.append(learn.recorder.values[-1][-1])
    print(f'Average accuracy for "{name}": {sum(accuracies) / num_runs}')

## Baseline performance  

Before getting carried away with normalization, we have to first know where we stand. Setting a baseline without normalizations means we can later evaluate the impact of normalization. Else we cannot know if normalization helped at all.

In [ ]:
# # data without normalization
# dls = get_dls(batch_tfms=[audio2spec])
# # run training loops
# train_loops(dls, name='No Norm')

## Performance with global statistics 
Next we normalize each audio waveform and the spectrograms with scalar global statistics. 

In [ ]:
# # data with waveform and global normalization
# dls = get_dls(item_tfms=[AudioNormalize],
#               batch_tfms=[audio2spec, GlobalSpecNorm])
# # run training loops
# train_loops(dls, name='Global Norm')

## Performance with channel statistics  
Finally, we normalize each audio waveform and the spectrograms with channel-based statistics.  

In [ ]:
# # get data with waveform and channel normalization
# dls = get_dls(item_tfms=[AudioNormalize],
#               batch_tfms=[audio2spec, ChannelSpecNorm])
# # run training loops
# train_loops(dls, name='Channel Norm')

# Conclusions

In this post we explored some issues around normalizing spectrograms for neural network training. We saw how spectrograms are fundamentally different than natural images and how there are at least two ways of normalizing them. 

We then implemented these two normalization techniques and tested them against a baseline in the `fastaudio` ESC-50 competition. 

There was a noticeable gain from using the global type of normalization, and a moderate gain from using the channel-based normalization. This makes intuitive sense, since a simple `show_batch` showed how varied these spectrograms were. In other words, there is a large amount of intra and inter channel variability both within and across classes. Under these conditions we'd expect that a more general, global normalization better suits this task. If the samples all came from a consistent source, say speech spectrograms, then the per-channel normalization might fare better.  

However, at the end of the day, there is no single theoretically correct way to normalize spectrograms for deep neural networks. Like many aspects of this field, the design choices will be experimental and depend on both the domain and problem specifics.  

I hope this post gave you a good notion for how to normalize spectrograms. I also hope it gave you some ideas, and potential approaches to try yourself! The ESC-50 challenge is an excellent playground to try them out.

# For fun, going as high as we can based on ImageNette

In [ ]:
# from fastai.basics import *
# from fastai.vision.all import *
# from fastai.callback.all import *
# from fastai.distributed import *
# from fastprogress import fastprogress
# from torchvision.models import *
# from fastai.vision.models.xresnet import *
# from fastai.callback.mixup import *
# from fastscript import *

# auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
#                  get_x=ColReader("filename", pref=path/"audio"), 
#                  splitter=CrossValidationSplitter(fold=1),
#                  item_tfms = [AudioNormalize], 
#                  batch_tfms = [audio2spec, GlobalSpecNorm],
#                  get_y=ColReader("category"))
# dbunch = auds.dataloaders(df, bs=64)

# # with mixup, train for longer
# epochs = 80

# # ranger optimizer 
# lr      = 2e-3
# mom     = 0.9
# sqrmom  = 0.99
# eps     = 1e-6
# beta    = 0
# opt_func = partial(ranger, mom=mom, sqr_mom=sqrmom, eps=eps, beta=beta)

# # default pooling
# pool = AvgPool

# # mish activation
# act_fn = Mish

# # loss function
# loss_func = LabelSmoothingCrossEntropyFlat()

# # with mixup augmentation
# mixup = True
# alpha = 0.4
# cbs = MixUp(alpha) if mixup else []

# # whether to use self-attention
# sa  = False
# sym = False

# # weight decay
# wd = 1e-2

# # the context manager way of dp/ddp, both can handle single GPU base case.
# gpu = 0
# n_gpu = torch.cuda.device_count()
# # ctx = learn.parallel_ctx if gpu is None and n_gpu else learn.distrib_ctx
# # ctx = learn.distrib_ctx

# # # model
# # n_out = 50
# # m = xse_resnext18
# # model = m(n_out=n_out, c_in=1, act_cls=act_fn, sa=sa, sym=sym, pool=pool)

# accuracies = []
# for run in range(num_runs):
#     print(f'Run: {run}')
# #     learn = Learner(dbunch, , opt_func=opt_func, \
# #             metrics=[accuracy], loss_func=loss_func)
#     learn = cnn_learner(dbunch, 
#                         xse_resnext18,
#                         pretrained=False,
#                         config=cnn_config(n_in=1),
#                         loss_fn=CrossEntropyLossFlat,
#                         opt_func=opt_func,
#                         metrics=[accuracy])
#     ctx = learn.parallel_ctx if gpu is None and n_gpu else learn.distrib_ctx
#     with partial(ctx, gpu)(): # distributed traing requires "-m fastai.launch"
#         print(f"Training in {ctx.__name__} context on GPU {gpu if gpu is not None else list(range(n_gpu))}")
#         learn.fit_flat_cos(epochs, lr, wd=wd, cbs=cbs)
# #         learn.fine_tune(epochs, lr, wd=wd, cbs=cbs)
#     accuracies.append(learn.recorder.values[-1][-1])

# print(f'Average accuracy for ImageNet training: {sum(accuracies) / num_runs}')